# Introduction
1. Intilizing the Data and loading Llama 2 piplline for text-genration
2. Defining Dataset Class
3. Summary Generation
4. Evalution
5. Results and conclusion

### Intilizing the Data and loading Llama 2 piplline for text-genration

In [9]:
import pandas as pd
import transformers
import torch
conv_data = pd.read_csv('dataset/100_conversation.csv')

In [10]:
conv_data.head()

,conversation,conv_id
0,Are you a fan of Google or Microsoft?\nBoth ar...,1
1,do you like dance?\nYes I do. Did you know Br...,2
2,Hey what's up do use Google very often?I reall...,3
3,Hi! do you like to dance?\nI love to dance a ...,4
4,do you like dance?\nI love it. Did you know Br...,5


In [11]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto" # if you have GPU
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


### Defining Dataset Class

In [43]:
from torch.utils.data import Dataset
# Make the prompt into the standard format
class summary_dataset(Dataset):
    
    def __init__(self,data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return f'For the convertation summarize in a paragraph it such that all the\
        important points are covered the conversation is as follows ```{self.data.conversation[idx]}``` \n Summary:'
dataset = summary_dataset(conv_data)

In [57]:
from tqdm import tqdm
import ipdb

### Summary Generation

In [60]:
coversations,summarizations = [],[]
for idx in tqdm(range(len(dataset)),desc="Generation: "):
    # print(idx)
    out = dataset[idx]
    
    sequences = pipeline(out,
                        do_sample=True,
                        top_k=10,
                        top_p = 0.9,
                        temperature = 0.2,
                        num_return_sequences=1,
                        eos_token_id=tokenizer.eos_token_id,
                        max_length=2048, # can increase the length of sequence 
                        )
    # ipdb.set_trace()
    conv,summ = sequences[0]['generated_text'].split("\n Summary:")
    # print(conv,summ)
    coversations.append(conv)
    summarizations.append(summ)

Generation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:10<00:00,  6.11s/it]


In [61]:
summ = pd.DataFrame({"conversations":coversations,"summarizations":summarizations})
summ.to_csv("dataset/summaries.csv",index=False)

### Evalutions

The challange with evalution is that we do not have ground truth summaries so we have calculated the metrics between the conversation and the generated summaries which is not ideal because they are overlap based metrics i.e rouge and chrf and hence they will suffer but metrics like bertscore would be more meaningful as they evalute sematices similarity between the source and the conversation.

In [1]:
import pandas as pd

In [2]:
tab = pd.read_csv("dataset/summaries.csv")

In [6]:
from evaluate import load
bertscore = load("bertscore")

In [7]:
from tqdm import tqdm
# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]
def cal():
    precision,recall,f1 = 0,0,0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = bertscore.compute(predictions=[b['summarizations']], references=[b['conversations']], lang="en")
        # print(out)
        precision += out['precision'][0]
        recall += out['recall'][0]
        f1 += out['f1'][0]
        
    print(f"The bertscore based scores \n Precision: {precision/len(tab)} \n Recall: {recall/len(tab)} \n F1: {f1/len(tab)}")

In [30]:
cal()

100it [00:06, 15.58it/s]

The bertscore based scores 
 Precision: 0.8656682378053665 
 Recall: 0.7922304481267929 
 F1: 0.8272823590040207


In [43]:
chrf = load("chrf")
def cal():
    bleu_s = 0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = chrf.compute(predictions=[b['summarizations']], references=[b['conversations']])
        # print(out)
        bleu_s += out['score']
        
    print(f"The chrf score: {bleu_s/len(tab)}")
cal()

100it [00:01, 94.99it/s]

The chrf score: 18.893789145698715


In [8]:
rouge = load('rouge')

In [17]:
def cal():
    precision,recall,f1 = 0,0,0
    for a,b in tqdm(tab.iterrows()):
        # print(b['conversations'],b['summarizations'])
        out = rouge.compute(predictions=[b['summarizations']], references=[b['conversations']])
        # print(out)
        precision += out['rouge1']
        # recall += out['rouge2']
        f1 += out['rougeL']
        
    print(f"The Rouge based scores \n rouge1: {precision/len(tab)} \n rougeL: {f1/len(tab)}")

In [18]:
cal()

100it [00:21,  4.76it/s]


The Rouge based scores 
 rouge1: 0.24814070160621415 
 rougeL: 0.16468335879954318


### Results and conclusion

LLMs like llama are well suited for the task open ended summarization and with zero and few shot prompting. In the experiment above we have use zero short prompting. As out task is open ended generation and no ground truth summaries are aviable to evalute our generated output. I decided to check the sematice and sentatic similarity between the generating summaries and the referce coversation which was summaried. Below I have reported the metrics for evalution.

|Metric|precision|recall|f1|
|-|---------|-----|--|
|Bertscore|0.8656|0.7922|0.8272|

We observe a good bertscore of 0.82 which indicates that the summaries were able to caputre the sematice of the conversations. but the same can not be said for rouge and chrf scores as these are based on n-gram overlap which is not ideal metric to use between the generating summaries and the referce coversation which was summaried.

```Rouge1: 0.2481```

```RougeL: 0.1646```

```CHRF: 18.89```

### Future work 
1. Few Sort Promptig can be tried to guide the model into generating specific kind of summary 
2. Summary Generation from GPT-4 and fine tune on llama2 with QLORA this will be one of the ways to distill the knowledge of highly capable model like GPT-4 to a much smaller model for a specific use case and as we are using QLORA we will can use the sample model with just differect adapter weights which are very small. 
3. Desing better evalutation statergy like NER based overlap between the generation and the converstion and if we have the time and resouces we should do some evalution with humans